In [1]:
from dotenv import load_dotenv
import os

from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [2]:
load_dotenv(dotenv_path='.env')
groq_api_key = os.getenv("GROQ_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
template = """
Qual a capital do Brasil ?
"""
prompt = ChatPromptTemplate.from_template(template=template)
print(prompt)

input_variables=[] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nQual a capital do Brasil ?\n'), additional_kwargs={})]


In [5]:
#llm = ChatGroq(model="llama3-8b-8192")
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
response = chain.invoke({"input":""})
print(response)

A capital do Brasil é Brasília. Foi inaugurada em 21 de abril de 1960 e foi planejada para ser a capital do país, com o intuito de promover o desenvolvimento do interior do Brasil.


FEW SHOT PROMPT

In [6]:
# Few Shot Prompt - provide a few examples in the template
TEMPLATE = """
Interprete o texto e faça uma avaliação.
sentimento: o texto expressa um sentimento positivo, neutro ou negativo?
tema: Qual o assunto do texto? Use exatamente uma palavra.

texto: {input}

Exemplos:
texto: O time fez uma excelente apresentação e marcou vários gols.
sentimento: positivo
tema: time

texto: O restaurante tem um serviço ruim e a comida deixa da desejar.
sentimento: negativo
tema: restaurante

texto: O podcast tem uma participação razoável mas sem grandes destaques.
sentimento: neutro
tema: podcast

"""

In [9]:
prompt = PromptTemplate(template=TEMPLATE)
# prompt = ChatPromptTemplate.from_template(template=TEMPLATE)
#llm = ChatGroq(model="llama3-8b-8192")
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
response = chain.invoke({"input":"O debate foi incrível com muitos destaques"})
print(response)

texto: O debate foi incrível com muitos destaques  
sentimento: positivo  
tema: debate


CHAIN OF THOUGTHS PROMPT

In [11]:
TEMPLATE = """

Responda a pergunta
Pergunta: {input}

Examplos:
Pergunta: "Se há 5 maçãs e você dá 2 para o João e 1 para a Maria, quantas maçãs restam?"
Explicação: "1. Começamos com 5 maçãs.
2. Damos 2 maçãs para o João. Agora restam 5 - 2 = 3 maçãs.
3. Em seguida, damos 1 maçã para a Maria. Agora restam 3 - 1 = 2 maçãs.
Resposta: "Restam 2 maçãs."

Pergunta: "Pergunta: "João é mais alto que Maria. Maria é mais alta que Ana. Quem é o mais baixo?"
Explicação: "1. João é mais alto que Maria.
2. Maria é mais alta que Ana.
3. Isso significa que Ana é mais baixa que Maria e João.
Resposta: "Ana é a mais baixa."

Pergunta: "Por que o Império Romano caiu?"
Explicação: "1. Uma das principais razões foi a invasão de tribos bárbaras, como os godos e os visigodos, que enfraqueceram o controle do império.
2. Outra razão foi a corrupção interna e má gestão, o que levou a uma economia instável.
3. A divisão do Império Romano em oriental e ocidental também enfraqueceu a coesão e a defesa do território."
Resposta: "A queda do Império Romano foi causada por uma combinação de invasões bárbaras, corrupção interna e a divisão do império.

"""


In [12]:
prompt = PromptTemplate(template=TEMPLATE)
# prompt = ChatPromptTemplate.from_template(template=TEMPLATE)
llm = ChatGroq(model="llama3-8b-8192")
#llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai_api_key)
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
response = chain.invoke({"input":"Por que o ceu é azul ?"})
print(response)

Pergunta: Por que o céu é azul?

Explicação:

1. O céu é composto principalmente por gases atmosféricos, como nitrogênio, oxigênio e argônio.
2. Entre esses gases, há um tipo de molécula chamada dióxido de carbono, que é responsável pela absorção de luz solar.
3. Quando a luz solar entra na atmosfera, é absorvida pelas moléculas de dióxido de carbono e outras substâncias presentes na atmosfera.
4. No entanto, a luz azul é absorvida menos pela atmosfera do que as outras cores, como o vermelho e o amarelo.
5. Isso significa que a luz azul é refletida de volta para nossos olhos, fazendo com que o céu apareça azul.

Resposta: O céu é azul porque a atmosfera absorve mais luz de outras cores, como o vermelho e o amarelo, e reflete mais a luz azul.


COMBINED PROMPTS

In [18]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

# Introduction
introduction_template = """
Interprete o texto e avalie o texto. Determine se o texto tem um sentimento positivo, neutro ou negativo. Além disso, identifique o assunto do texto 
em uma palavra."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

# Example
example_template = """
Vamos começar avaliando a sentença. O texto "{example_text}" tem um sentimento {example_sentiment}. Coloque a resposta no formato normal: "{example_format}" """
example_prompt = PromptTemplate.from_template(example_template)

# Execution
execution_template = """
Agora, faça o mesmo para o seguinte texto: "{input}".
"""
execution_prompt = PromptTemplate.from_template(execution_template)


# Composing the full prompt
full_template = """{introduction}

{example}

{execution}"""
full_prompt = PromptTemplate.from_template(full_template)

# PipelinePrompts
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("execution", execution_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)


In [19]:
model = ChatOpenAI(model="gpt-4o-mini",openai_api_key=openai_api_key)
output_parser = StrOutputParser()
chain = pipeline_prompt | model | output_parser
response = chain.invoke({
    "example_text": "O restaurante estava maravilhoso, com uma vista deslumbrante e uma comida de alta qualidade.",
    "example_sentiment": "positivo",
    "example_format": "{'Sentimento': 'positivo', 'Tema': 'restaurante'}",
    #"example_format": "Sentimento: positivo, Tema: restaurante",
    "input": "O tempo não está bom, parece que vai cair uma tempestade."
})
print(response)


{'Sentimento': 'negativo', 'Tema': 'tempo'}
